In [ ]:
from cuopt import routing
import cudf

# Heterogenous Fleet Routing
## Pickup and Delivery Problem with Time Windows using Heterogenous Fleet of Vehicles

In scenarios such as food delivery, the delivery fleet may consist of various types of vehicles, for example bikes, and cars, and each type of vehicle has own advantages and limitations. For example, in crowded streets of NYC, it might be faster to reach a nearby destination on bike compared to car, while it is much faster with car in suburban areas. Service provides can improve customer satisfaction, reduce costs, and increase earning opportunity for drivers, using various types of vehicles depending on the geography of the service area.  

### Problem Details:
- 5 customer orders each with an associated demand, merchant, and time windows 
- 2 merchants 
- 3 vehicles 
  - 2 bikes 
  - 1 car
- 7 locations in total (5 customers + 2 merchants)

### Cost Matrix

#### Define cost matrix for each vehicle type

In [ ]:
# locations 0, 1 correspond to merchant 
# locations 2 to 6 correspond to customers
n_locations = 7
time_matrix_car = cudf.DataFrame([
    [0., 6.,  5., 6., 6., 7., 4.], 
    [6., 0.,  3., 8., 7., 3., 2.], 
    [5., 3.,  0., 4., 11., 6., 4.], 
    [6., 8.,  4., 0., 12., 11., 10.], 
    [6., 7., 11., 12., 0., 7., 4.],
    [7., 3.,  6., 11., 7., 0.,  3.], 
    [4., 2.,  4., 10., 4., 3.,  0.]])

time_matrix_bike = cudf.DataFrame([
    [0., 15., 10.,  9.,  10.,  21., 6.],
    [15., 0.,  6.,  15.,  12., 5., 4.], 
    [10., 6.,  0.,   9.,  20, 12., 9.],
    [9., 15.,  9.,  0., 20., 22., 20.],
    [10.,12., 20., 20., 0., 15., 8.], 
    [21., 5., 12., 22., 15., 0., 8.], 
    [6.,  4., 9., 20., 8.,  8.,  0.]])

print('time matrix for bike:: \n', time_matrix_bike, '\n')
print('time matrix for car:: \n', time_matrix_car, '\n')

### Fleet Data

Set up mixed fleet data

In [ ]:
n_vehicles = 3

# type 0 corresponds to bike and type 1 corresponds to car
vehicle_types = cudf.Series([0, 0, 1])

# bikes can carry two units of goods while car can carry 5 units of goods
vehicle_capacity = cudf.Series([2, 2, 5])

print(n_vehicles)
print(vehicle_types)
print(vehicle_capacity)

### Customer Orders

Setup Customer Order Data

The customer order data contains the information of merchant, the amount of goods, and the time window for the delivery.

In [ ]:
customer_order_data = cudf.DataFrame({
    "pickup_location":       [0,  0,  1,  1,  1],
    "delivery_location":     [2,  3,  4,  5,  6],
    "order_demand":          [1,  2,  2,  1,  2],
    "earliest_pickup":       [0,  0,  0,  0,  0],
    "latest_pickup":         [30, 120, 60, 120, 45],
    "pickup_service_time":   [10,  10,  10,  10,  10],
    "earliest_delivery":     [0,  0,  0,  0,  0],
    "latest_delivery":       [30, 120, 60, 120, 45],
    "delivery_serivice_time":[10,  10,  10,  10,  10]
})
customer_order_data

### cuOpt routing DataModel

#### Initialize routing.DataModel object

In [ ]:
# a pickup order and a delivery order are distinct
n_orders = len(customer_order_data) * 2

data_model = routing.DataModel(n_locations, n_vehicles, n_orders)

#### Set Vehicle types and corresponding cost matrices

In [ ]:
# set matrices associated with each vehicle type
data_model.set_vehicle_types(vehicle_types)

data_model.add_cost_matrix(time_matrix_bike, 0)
data_model.add_cost_matrix(time_matrix_car, 1)

#### Setting Order locations
From the cuOpt solver_settings perspective, each distinct transaction (pickup order or delivery order) is treated separately. The locations for each order is specified using order locations. The first entry in order locations array is always reserved for the notion of depot for the problem. So for a total n orders, the order location array is of size 2n+1.  

In [ ]:
pickup_order_locations = customer_order_data['pickup_location'] 
delivery_order_locations = customer_order_data['delivery_location']
order_locations = cudf.concat([pickup_order_locations, delivery_order_locations], ignore_index=True)

print(order_locations)

# add order locations
data_model.set_order_locations(order_locations)

#### Mapping pickups to deliveries
Order locations do not provide information regarding the type of order (i.e, pickup or delivery). This information is provided to solver by setting two arrays pickup_orders and delivery_orders. The entries of these arrays corresponding the order numbers in exanded list described above. 

For a pair order i, pickup_orders[i] and delivery_orders[i] correspond to order index in 2n total orders. Furthermore, order_locations[pickup_orders[i]] and order_locations[delivery_orders[i]] indicate the pickup location and delivery location of order i. 

In [ ]:
npair_orders = int(len(order_locations)/2)
pickup_orders = cudf.Series([i for i in range(npair_orders)])
delivery_orders = cudf.Series([i + npair_orders for i in range(npair_orders)])

# add pickup and delivery pairs.
data_model.set_pickup_delivery_pairs(pickup_orders, delivery_orders)


#### Set the per order demand

From the perspective of the cuOpt solver, each distinct transaction (pickup order or delivery order) are treated separately with demand for pickup denoted as positive and the corresponding delivery treated as negative demand.

In [ ]:
# This is the number of goods that need to be delivered
raw_demand = customer_order_data["order_demand"]

# When dropping off goods, remove one unit of demand from the vehicle
drop_off_demand = raw_demand * -1

# Create pickup and delivery demand
order_demand = cudf.concat([raw_demand, drop_off_demand], ignore_index=True)

order_demand

# add the capacity dimension
data_model.add_capacity_dimension("demand", order_demand, vehicle_capacity)

#### Time Windows

Create per order time windows similar to demand.  Set earliest time and service time of depot to be zero and the latest time to be a large value so that all orders are fulfilled. 

In [ ]:
# create earliest times
order_time_window_earliest = cudf.concat([customer_order_data["earliest_pickup"], customer_order_data["earliest_delivery"]], ignore_index=True)

# create latest times
order_time_window_latest = cudf.concat([customer_order_data["latest_pickup"], customer_order_data["latest_delivery"]], ignore_index=True)

# create service times
order_service_time = cudf.concat([customer_order_data["pickup_service_time"], customer_order_data["delivery_serivice_time"]], ignore_index=True)

# add time window constraints
data_model.set_order_time_windows(order_time_window_earliest, order_time_window_latest, order_service_time)

# set time windows for the fleet
vehicle_earliest_time = cudf.Series([0] * n_vehicles)
vehicle_latest_time = cudf.Series([1000] * n_vehicles)

data_model.set_vehicle_time_windows(vehicle_earliest_time, vehicle_latest_time)

### CuOpt SolverSettings

Set up routing.SolverSettings.

In [ ]:
solver_settings = routing.SolverSettings()

# solver_settings will run for given time limit.  Larger and/or more complex problems may require more time.
solver_settings.set_time_limit(0.05)

### Solution

In [ ]:
routing_solution = routing.Solve(data_model, solver_settings)
if routing_solution.get_status() == 0:
    print("Cost for the routing in time: ", routing_solution.final_cost)
    print("Vehicle count to complete routing: ", routing_solution.vehicle_count)
    print(routing_solution.route)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", routing_solution.get_status())

_____

#### SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

#### SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the "Software"),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the
Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
DEALINGS IN THE SOFTWARE.

---